In [1]:
import pandas as pd
import numpy as np
import h5py
import matplotlib.pyplot as plt

In [2]:
def window(iterable, size=8192):
    i = iter(iterable)
    win = []
    for e in range(0, size):
        win.append(next(i))
    yield win
    for e in i:
        win = win[1:] + [e]
        yield win

In [3]:
def get_data(f):
    df = pd.DataFrame()
    for x in f.values():
        dset = f[x.name].items()
        data = pd.DataFrame(dset)
        data = data.drop([0,1,2,3]).sort_values(0)
        data.reset_index(drop = True, inplace = True)
        list = data[1]
        for index, l in enumerate(list):
            arr = np.zeros((27421,3), dtype='float64')
            l.read_direct(arr)
            norm = pd.DataFrame(arr).drop(0,axis=1).rename(index=str, columns={1: 'coluna_1', 2: 'coluna_2'})
            norm['coluna_1'] = norm['coluna_1'].div(norm['coluna_1'].max())
            norm['coluna_2'] = norm['coluna_2'].div(norm['coluna_2'].max())
            list[index] = norm
        data = data.rename(index=str, columns={1: x.name})
        df[x.name] = data[x.name]
    noise = df.copy()
    for i in range(0,len(df.columns)):
        for j in range(0,len(df[df.columns[i]])):
            df[df.columns[i]][j].reset_index(drop = True, inplace = True)
            noise[noise.columns[i]][j] = pd.DataFrame(0.5*((np.random.rand(len(df[df.columns[i]][j]),2)*2)-1),columns=['coluna_1','coluna_2'])
            df[df.columns[i]][j] = df[df.columns[i]][j].add(noise[noise.columns[i]][j])
    return df, noise

In [4]:
path = 'rh_FiniteRadii_CodeUnits'
f = h5py.File('../dados/'+path+'.h5', 'r')

In [ ]:
df,noise = get_data(f)

In [ ]:
df['/R0100.dir'][0]['coluna_1'].plot()

In [6]:
dados = pd.DataFrame(list( window( df['/R0100.dir'][0]['coluna_1'].values, 8192 ) ))

In [7]:
ruidos = pd.DataFrame(list( window( noise['/R0100.dir'][0]['coluna_1'].values, 8192 ) ))

In [8]:
dds = dados[len(dados)-(len(dados)/8):]

In [9]:
rds = ruidos[len(dados)-(len(dados)/8):]

In [10]:
x = pd.DataFrame(np.ones((len(dds),2)), dtype=int, columns=['A','B'])
zeros = pd.DataFrame(np.zeros(len(dds), dtype=int))
x['B'] = zeros[0]

In [11]:
y = pd.DataFrame(np.zeros((len(rds),2)), dtype=int, columns=['A','B'])
ones = pd.DataFrame(np.ones(len(rds), dtype=int))
y['B'] = ones[0]

In [12]:
dataframe = dds.append(rds,ignore_index=True)

In [13]:
target = x.append(y,ignore_index=True)

In [14]:
dataframe.to_csv('dados 0001.csv')

In [15]:
target.to_csv('target 0001.csv')